In [67]:
import tarfile
import pandas as pd
import json
import numpy as np
import ast
import os.path
from pathlib import Path

In [84]:
path = "D:\Projects\YELP\data\yelp_dataset.tar"
dataPath = r"D:/Projects/YELP/data/"
# dataPath = os.path.join("D:", "Projects","YELP","Data")
# dataPath = Path(r"D:\Projects\YELP\data\")

In [ ]:
# open file
file = tarfile.open(path)

In [ ]:
# extracting file
file.extractall()

file.close()

In [3]:
def split_times(times):
    if pd.isnull(times):
        return None, None
    times = times.split('-')
    start_time = times[0]
    end_time = times[1]
    return start_time, end_time


def createHoursFile(hoursDF):
    main_hours = []
    for index, rows in hoursDF.iterrows():
        subdict = {'business_id':None,
                   'Monday_Open':None, 'Monday_Close':None,
                   'Tuesday_Open':None, 'Tuesday_Close':None,
                   'Wednesday_Open':None, 'Wednesday_Close':None,
                   'Thursday_Open':None, 'Thursday_Close':None,
                   'Friday_Open':None, 'Friday_Close':None,
                   'Saturday_Open':None, 'Saturday_Close':None,
                   'Sunday_Open':None, 'Sunday_Close':None}
        busID = rows['business_id']
        subdict['business_id'] = busID
        hRow = rows['hours']
        if (pd.isnull(hRow)) or (hRow == 'null'):
            main_hours.append(subdict)
        else:
#             hRow = ast.literal_eval(hRow)
            for k,v in hRow.items():
                if k == 'Monday':
                    openH, closeH = split_times(v)
                    subdict['Monday_Open'] = openH
                    subdict['Monday_Close'] = closeH
                if k == 'Tuesday':
                    openH, closeH = split_times(v)
                    subdict['Tuesday_Open'] = openH
                    subdict['Tuesday_Close'] = closeH
                if k == 'Wednesday':
                    openH, closeH = split_times(v)
                    subdict['Wednesday_Open'] = openH
                    subdict['Wednesday_Close'] = closeH
                if k == 'Thursday':
                    openH, closeH = split_times(v)
                    subdict['Thursday_Open'] = openH
                    subdict['Thursday_Close'] = closeH
                if k == 'Friday':
                    openH, closeH = split_times(v)
                    subdict['Friday_Open'] = openH
                    subdict['Friday_Close'] = closeH
                if k == 'Saturday':
                    openH, closeH = split_times(v)
                    subdict['Saturday_Open'] = openH
                    subdict['Saturday_Close'] = closeH
                if k == 'Sunday':
                    openH, closeH = split_times(v)
                    subdict['Sunday_Open'] = openH
                    subdict['Sunday_Close'] = closeH
            main_hours.append(subdict)
    hoursDF = pd.DataFrame(main_hours)
    return hoursDF

In [45]:
def clean_values(value):
    if pd.isnull(value):
        return
    if type(value) == bool:
        return value
    if value.startswith("u'"):
        value = value.replace("u'", "").replace("'", "")
        return value
    elif value.startswith("'"):
        value = value.replace("'", "")
        return value
    else:
        return value



def create_new_details(attDict, current_column):
    if (attDict == 'None') or (pd.isnull(attDict)):
        return
    attDict = ast.literal_eval(attDict)
    attDictApp = {}
    for k1, v1 in attDict.items():
        v1 = clean_values(v1)
        k1 = current_column + '_' + k1
        attDictApp[k1] = v1
#     print(attDictApp)
    return attDictApp


def createAttributesData(attSub):
    attDF = []
    expander = ['BusinessParking', 'Ambience', 'GoodForMeal', 'Music', 'DietaryRestrictions', 'BestNights', 'HairSpecializesIn']
    for ind, rows in attSub.iterrows():
        subdict = {}
        busID = rows['business_id']
        subdict['business_id'] = busID
        attsDict = rows['attributes']
        if (pd.isnull(attsDict)) or (attsDict == 'null'):
            attDF.append(subdict)
        else:
            for k,v in attsDict.items():
                if k in expander:
                    expandedAttrs = create_new_details(v, k)
                    if type(expandedAttrs) == dict:
                        subdict.update(expandedAttrs)
                else:
                    subdict[k] = v
            attDF.append(subdict)
    attributesDF = pd.DataFrame(attDF)
    attributesDF['WiFi'] = attributesDF['WiFi'].apply(clean_values)
    attributesDF['Alcohol'] = attributesDF['Alcohol'].apply(clean_values)
    attributesDF['RestaurantsAttire'] = attributesDF['RestaurantsAttire'].apply(clean_values)
    attributesDF['NoiseLevel'] = attributesDF['NoiseLevel'].apply(clean_values)
    attributesDF['Smoking'] = attributesDF['Smoking'].apply(clean_values)
    attributesDF['BYOBCorkage'] = attributesDF['BYOBCorkage'].apply(clean_values)
    attributesDF['AgesAllowed'] = attributesDF['AgesAllowed'].apply(clean_values)
    return attributesDF

In [48]:
def convert_business():
    lines = []
    with open(dataPath + "yelp_academic_dataset_business.json", encoding="latin1") as file:
#     with open("D:\Projects\YELP\data\yelp_academic_dataset_business.json", encoding="latin1") as file:
        for line in file:
            line = line.replace('\/', ' ').replace('\n', '').replace(':null', ': "null"')
            lines.append(line)
#     lines = lines[:10000]
    evaled = []
    for i, line_ in enumerate(lines):
        if i%10000 == 0:
            print(i)
        try:
            newline = ast.literal_eval(line_)
            evaled.append(newline)
        except Exception as e:
            print(line_)
            print(e)
            break
    df = pd.DataFrame(evaled)
    print(df.info())
        
    business = df[['business_id', 'name', 'address', 'city', 'state', 'postal_code', 
                   'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'categories']]

    hours = df[['business_id', 'hours']]

    attributes = df[['business_id', 'attributes']]
    
    print('Doing Business')
    business.to_excel(dataPath + "businessOnly.xlsx", index=False)
#     business.to_excel("D:\Projects\YELP\data\businessOnly.xlsx", index=False)
    
    print('Doing Hours')
    hoursDF = createHoursFile(hours)
    hoursDF.to_csv(dataPath + "businessHours.csv", index=False)
#     hoursDF.to_csv("D:\Projects\YELP\data\businessHours.csv", index=False)
    
    print('Doing Attributes')
    attributesDF = createAttributesData(attributes)
    attributesDF.to_csv(dataPath + "businessAttributesNew.csv", index=False)
#     attributesDF.to_csv("D:\Projects\YELP\data\businessAttributesNew.csv", index=False)
    return attributesDF

In [49]:
test = convert_business()

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    150346 non-null  object 
 12  categories    150346 non-null  object 
 13  hours         150346 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB
None
Doing Business
Doin

In [31]:
test.head()

,business_id,ByAppointmentOnly,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,CoatCheck,RestaurantsTakeOut,RestaurantsDelivery,Caters,WiFi,...,Open24Hours,RestaurantsCounterService,AgesAllowed,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian
0,Pns2l4eNsfO8kk83dixA6A,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,False,True,True,2,False,False,False,False,u'no',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,False,False,True,1,NaN,True,False,True,u'free',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,NaN,True,True,NaN,NaN,True,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def convert_user():
    lines = []
    with open(dataPath + "yelp_academic_dataset_user.json", encoding="latin1") as file:
#     with open("D:\Projects\YELP\data\yelp_academic_dataset_user.json", encoding="latin1") as file:
        for line in file:
            line = line.replace('\/', ' ').replace('\n', '').replace(':null', ': "null"')
            lines.append(line)
    evaled = []
    for i, line_ in enumerate(lines):
        if i%50000 == 0:
            print(i)
        try:
            newline = ast.literal_eval(line_)
            evaled.append(newline)
        except Exception as e:
            print(line_)
            print(e)
            break
    df = pd.DataFrame(evaled)
    alldf = df.drop(columns = ['friends'], axis=1)
    alldf.to_csv(dataPath + "user.csv", index=False)
    alldf.to_csv(dataPath + "user.csv", index=False)
    
    friendsdf = df[['user_id', 'friends']]
    friendsdf.to_csv(dataPath + "friends.csv", index=False)
#     friendsdf.to_csv("D:\Projects\YELP\data\friends.csv", index=False)

In [86]:
def convert_tip():
    lines = []
    with open(dataPath + "yelp_academic_dataset_tip.json", encoding="latin1") as file:
#     with open("D:\Projects\YELP\data\yelp_academic_dataset_tip.json", encoding="latin1") as file:
        for line in file:
            line = line.replace('\/', ' ').replace('\n', '').replace(':null', ': "null"')
            lines.append(line)
    evaled = []
    for i, line_ in enumerate(lines):
        if i%50000 == 0:
            print(i)
        try:
            newline = ast.literal_eval(line_)
            evaled.append(newline)
        except Exception as e:
            print(line_)
            print(e)
            break
    df = pd.DataFrame(evaled)
    df.to_csv(dataPath + "tip.csv", index=False)
#     df.to_csv("D:\Projects\YELP\data\tip.csv", index=False)

In [65]:
def convert_review():
    lines = []
    with open(dataPath + "yelp_academic_dataset_review.json", encoding="latin1") as file:
#     with open("D:\Projects\YELP\data\yelp_academic_dataset_review.json", encoding="latin1") as file:
        for line in file:
            line = line.replace('\/', ' ').replace('\n', '').replace(':null', ': "null"')
            lines.append(line)
    evaled = []
    for i, line_ in enumerate(lines):
        if i%50000 == 0:
            print(i)
        try:
            newline = ast.literal_eval(line_)
            evaled.append(newline)
        except Exception as e:
            print(line_)
            print(e)
            break
    df = pd.DataFrame(evaled)
    df.to_csv(dataPath + "review.csv", index=False)
#     df.to_csv("D:\Projects\YELP\data\review.csv", index=False)

In [92]:
# convert_review()
# convert_tip()
# convert_user()

In [28]:
# test values outliers like 'monday': '0:0-10:30, 12:30-17:30'
for val in hours['hours']:
#     print(val)
#     print('---')
    if pd.notnull(val):
        val = ast.literal_eval(val)
        for k,v in val.items():
            vals = v.split(',')
            if len(vals) > 1:
                print(val)